In [58]:
#prepearing date for analisis
import pandas as pd
adress_control = pd.read_excel('качалкин + кнутов квартал.xlsx')
adress_control['Время открытия'] = adress_control['Время открытия'].fillna(method='ffill')
adress_control.to_excel('качалкин + кнутов квартал.xlsx')

In [35]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm
import requests
import time
import pandas as pd
from openpyxl import load_workbook

pd.set_option('display.max_columns', None, 'display.max_rows', None)
pd.options.mode.chained_assignment = None

adress_control = pd.read_excel('качалкин + кнутов квартал.xlsx')
adress_control.columns = adress_control.columns.str.lower()
adress_control.columns = adress_control.columns.str.strip()

#adress_control = adress_control.drop(adress_control[adress_control['тип заказа'] == 'На время самовывоз'].index)
#adress_control = adress_control.drop(adress_control[adress_control['тип заказа'] == 'Доставка самовывоз'].index)

counter = 0

def take_short_adress(x):
    per = ''
    if (len(x.split('д. ')[1].split()) > 1): 
        if (len(x.split('д. ')[1].split()[1]) == 1):
            per =  x.split('д. ')[1].split()[0] + x.split('д. ')[1].split()[1]
        else:
            per =  x.split('д. ')[1].split()[0]
    else:
        per = x.split('д. ')[1].split()[0]
    
    if ('корп./стр.' in list(x.split())):
        per = per + ' ' + x.split('корп./стр.')[1].split()[0]
     
    adress = x.split('д. ')[0] + 'д. ' + per
    return adress

def save_table_coord(df, sheet):
    writer = pd.ExcelWriter('courier_with_lat_and_lang.xlsx', engine='openpyxl')
    writer.book = load_workbook('courier_with_lat_and_lang.xlsx')   
    writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
    reader = pd.read_excel(r'courier_with_lat_and_lang.xlsx', sheet_name=sheet)
    df.to_excel(writer, sheet_name=sheet, index=False, header=False, startrow=len(reader)+1)
    writer.close() 

    
def take_coord1(x):
    DF_list_for_addition = list(x) + ['111'] 
    DF_for_addition = pd.DataFrame([DF_list_for_addition])
    save_table_coord(DF_for_addition, 'Sheet1')
    print(1)

def take_coord(x):
    global counter
    counter += 1 
    if counter < 999:
        key_api = '87101335-848d-4b24-8985-9b4be4756821'
    elif (counter >= 999) and (counter < 1998):
        key_api = '967d5b0c-1f61-4eea-83f7-888be714c3a2'
    else:
        key_api = 'edc75c02-4b15-4be0-98fa-76d1ad08dce4'
 
    time.sleep(1)
    response = requests.get("https://geocode-maps.yandex.ru/1.x/?apikey=" + key_api + "&geocode=" + x['адрес B SF'])
    soup = BeautifulSoup(response.text,'lxml')
    if (soup.find('pos')):
        DF_list_for_addition = list(x) + [soup.find('pos').text]
        DF_for_addition = pd.DataFrame([DF_list_for_addition])
        save_table_coord(DF_for_addition, 'Sheet1')
        return soup.find('pos').text
    else:
        DF_list_for_addition = list(x) + ['адреса ошибка']
        DF_for_addition = pd.DataFrame([DF_list_for_addition])
        save_table_coord(DF_for_addition, 'Sheet1')
        return 'адреса ошибка'

def add_gorod(x):
    if x['город'] == 'Орел':
        return 'г. Орел ' + x['адрес']
    else:
        return 'Орловская область Орловский район ' + x['город'] + ' ' +  x['адрес']

adress_control['время отправки доставки'].iloc[0].month
adress_control['день + месяц'] = adress_control['время отправки доставки'].apply(lambda x: str(x.day) + '.' + str(x.month) + '.' + str(x.year))
adress_control['день + месяц'].unique()
adress_control['адрес А'] = 0
adress_control['адрес В'] = adress_control.apply(add_gorod, axis=1) 

adress_control['адрес B SF'] = adress_control['адрес В'].apply(take_short_adress)
adress_control['coords B'] = adress_control.apply(take_coord, axis=1)

#adress_control['coords B'] = adress_control['coords B'].apply(lambda x: x.split()[1] + ' ' + x.split()[0])
#adress_control.to_excel('courier_with_lat_and_lang.xlsx')

In [51]:
# запись ошибочный адресов для контрля
adress_control = pd.read_excel('courier_with_lat_and_lang.xlsx')
writer = pd.ExcelWriter('ошибки адреса.xlsx', engine='openpyxl')
writer.book = load_workbook('ошибки адреса.xlsx')   
writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
reader = pd.read_excel(r'courier_with_lat_and_lang.xlsx')
df1 = adress_control[adress_control['coords B'] == 'адреса ошибка']
df1.to_excel(writer, index=False, header=False, startrow=len(reader)+1)
writer.close() 

In [52]:
# проверка ошибочных адресов
adress_control = pd.read_excel('courier_with_lat_and_lang.xlsx')

if adress_control[adress_control['coords B'] == 'адреса ошибка'].size == 0:
    print('Without errors')
else:
    print('Warning! Before exceed next module, please fix adress errors')
    display(adress_control[adress_control['coords B']== 'адреса ошибка'])

Without errors


In [ ]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm
import requests
import time
import pandas as pd
from openpyxl import load_workbook
import numpy as np 

t_points = ['52.951462 36.025452', '52.978591 36.075605', '53.014416 36.133214']

adress_control = pd.read_excel('courier_with_lat_and_lang.xlsx')
adress_control.columns = adress_control.columns.str.strip()
adress_control['адрес А LBA'] = 0
adress_control['coords B'] = adress_control['coords B'].apply(lambda x: x.split()[1] + ' ' + x.split()[0])
adress_control_concat_table = pd.DataFrame()
finish_courier_trip = pd.DataFrame(columns=['trip', 'courier', 'day'])
otchet_about_couriers = pd.DataFrame(columns=['Координаты', 'Маршрут','kourier','day'])

def create_mass(slice_list, slice_list_A_B, kourier, day):
    slice_list_temp = slice_list.copy()
    #display(slice_list_A_B[0])
    slice_list_temp_A_B = slice_list_A_B
    slice_list_full = []
    slice_list_full_A_B = []
    tt=0
    l = len(slice_list_temp)

    for i in range(l):
        if i==0: 
            tt = slice_list_temp[0][0]
            slice_list_full = slice_list_full + [slice_list_temp[0]]
            slice_list_full_A_B = slice_list_full_A_B + [slice_list_temp_A_B[0]]
            continue

        if slice_list_temp[i][0] in t_points:
            tt = slice_list_temp[i][0]
            slice_list_full = slice_list_full + [[slice_list_temp[i-1][1],slice_list_temp[i][0]]] 
            slice_list_full_A_B = slice_list_full_A_B + [[slice_list_temp_A_B[i-1][1], slice_list_temp_A_B[i][0]]] 
            
        slice_list_full = slice_list_full + [slice_list_temp[i]] 
        slice_list_full_A_B = slice_list_full_A_B + [slice_list_temp_A_B[i]] 
    #print(slice_list_temp)
    slice_list_full = slice_list_full + [[slice_list_temp[-1][1], tt]]
    
    if tt == '52.951462 36.025452':
        tt = 'Алроса'
    elif tt == '52.978591 36.075605':
        tt = 'Горького'
    elif tt == '53.014416 36.133214':
        tt = 'Раздольная'
    
    slice_list_full_A_B = slice_list_full_A_B + [[slice_list_temp_A_B[-1][1], tt]]
    display(slice_list_full_A_B)
    distance_array = create_route(slice_list_full, kourier, day)
    
    #display(slice_list_full)
    #display(distance_array)
    df_otchet = pd.DataFrame(list(zip(slice_list_full, slice_list_full_A_B, distance_array)), columns =['Координаты', 'Адреса' , 'Маршрут'])
    df_otchet['kourier'] = kourier 
    df_otchet['day'] = day 
    return df_otchet

def create_route(slice_list, kourier, day):
    global finish_courier_trip 
    browser = Chrome('/Users/user/kvikfud/chromedriver_win32/chromedriver')
    roat_distance = 0
    list_roat_distance = []
    distance_array = []
    #display(slice_list)
    for A,B in slice_list:
        
        e = ''
        delay = 4
        while e == '':
            try:
                url = "https://www.mapdevelopers.com/distance_from_to.php?&from=" + A + '&to=' + B
                browser.get(url)
                sleep(delay)
                distance_array.append(browser.find_element_by_id("driving_status").text.split()[5])
                e = ' '
                
            except NoSuchElementException:
                sleep(10)
                print('Connection refused. Retrying...')
                        
        roat_distance = roat_distance + float(browser.find_element_by_id("driving_status").text.split()[5])
    
    list_roat_distance = [roat_distance] + [kourier] + [day]
    finish_courier_trip = finish_courier_trip.append(pd.DataFrame([list_roat_distance], columns=['trip', 'courier', 'day']), ignore_index=True)
    list_roat_distance = pd.DataFrame([list_roat_distance], columns=['trip', 'courier', 'day'])
    save_table(list_roat_distance, 'Sheet1')
    #print(list_roat_distance)
    return distance_array

def save_table(df, sheet):
    writer = pd.ExcelWriter('Отчет по курьерам.xlsx', engine='openpyxl')
    writer.book = load_workbook('Отчет по курьерам.xlsx')   
    writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
    reader = pd.read_excel(r'Отчет по курьерам.xlsx', sheet_name=sheet)
    df.to_excel(writer, sheet_name=sheet, index=False, header=False, startrow=len(reader)+1)
    writer.close()  
    
for i in adress_control['день + месяц'].unique():
    slice_first = adress_control[adress_control['день + месяц']==i]
    
    for j in slice_first['курьер'].unique():
        slice_second = slice_first[slice_first['курьер']==j].sort_values(by=['фактическое время доставки'])
        slice_second['адрес А'].iloc[0] = slice_second['торговое предприятие'].iloc[0]
        slice_second['адрес А LBA'].iloc[0] = slice_second['торговое предприятие'].iloc[0]
        
        for z in range(len(slice_second)-1):
            if slice_second['фактическое время доставки'].iloc[z] < slice_second['время отправки доставки'].iloc[z+1]:
                slice_second['адрес А'].iloc[z+1] = slice_second['торговое предприятие'].iloc[z+1]
                slice_second['адрес А LBA'].iloc[z+1] = slice_second['торговое предприятие'].iloc[z+1]
        
        for z in range(len(slice_second)):
            if slice_second['адрес А'].iloc[z] == 0:
                slice_second['адрес А'].iloc[z] = slice_second['coords B'].iloc[z-1]
                slice_second['адрес А LBA'].iloc[z] = slice_second['адрес В'].iloc[z-1]
        
        #slice_second['адрес В'].iloc[len(slice_second)-1] = slice_second['торговое предприятие'].iloc[len(slice_second)-1]
        #slice_second['coords B'].iloc[len(slice_second)-1] = slice_second['торговое предприятие'].iloc[len(slice_second)-1]
                           
        adress_control_concat_table = pd.concat([adress_control_concat_table, slice_second], ignore_index=True)         

        slice_second['адрес А'] = slice_second['адрес А'].replace({'Алроса' : '52.951462 36.025452', 'Горького' : '52.978591 36.075605', 'Раздольная' : '53.014416 36.133214'})
        slice_second['coords B'] = slice_second['coords B'].replace({'Алроса' : '52.951462 36.025452', 'Горького' : '52.978591 36.075605', 'Раздольная' : '53.014416 36.133214'})
        slice_list = [list(a) for a in zip(slice_second['адрес А'].tolist(), slice_second['coords B'].tolist())]
        slice_list_A_B = [list(a) for a in zip(slice_second['адрес А LBA'].tolist(), slice_second['адрес В'].tolist())] 
        #display(slice_list)
        temp_value = create_mass(slice_list, slice_list_A_B, j, i)
        
        save_table(temp_value, 'Sheet2')
        #display(otchet_about_couriers)
        #otchet_about_couriers = pd.concat([otchet_about_couriers], temp_value, ignore_index=True)
                
#display(otchet_about_couriers)
#display(finish_courier_trip)
        


In [57]:
#Извлечение информации из файла с пробегом

import pandas as pd
dermo = pd.read_excel('д.xlsx')

for i in range(len(dermo)-1):
    if dermo['Сотрудник'].iloc[i] == 'а':
        dermo['Сотрудник'].iloc[i] = dermo['Сотрудник'].iloc[i-1]

for i in range(len(dermo)-1):
    if dermo['Сотрудник'].iloc[i] == 'р':
        dermo['Сотрудник'].iloc[i] = dermo['Сотрудник'].iloc[i-1]

dermo = dermo[dermo['Имя курьера']!=0]

dermo.to_excel('д.xlsx')